# Part 3: Annotating Regions of Interest with Napari

**Tutor:** Anthony Christidis
**Time:** 20 minutes

---

Welcome to the second half of the workshop! In the first part, Tim showed us how to load, explore, and plot `SpatialData` objects. Now, we're going to start using these tools to perform analysis.

A common first step in analyzing a tissue slide is to identify and isolate specific regions of interest (ROIs), such as a tumor area, a healthy region, or an immune infiltrate. In this notebook, we'll learn how to use `napari` to perform this critical task.

**Goals:**
1. Use `napari` to manually draw and annotate a region of interest (ROI).
2. Use the `polygon_query` function to programmatically select data within our annotated ROI.
3. Use this selection to create a new annotation in our `AnnData` table.

In [ ]:
%load_ext jupyter_black

import spatialdata as sd
import napari_spatialdata as nsd
import geopandas as gpd
from shapely.geometry import Polygon

data_path = "../data/"
sdata_visium = sd.read_zarr(data_path + "visium_glioblastoma_subset.zarr")

### Your Turn: Interactively Annotating a Region

We will now use `napari` to draw a polygon around a region that looks interesting. In the previous notebook, we saw that the gene `CST3` was highly expressed in one area. Let's try to delineate that area.

**Instructions:**
1. Run the cell below to launch `napari`.
2. In the `napari-spatialdata` viewer, add the `hires_image` and the `shapes` layers.
3. In the annotation panel, color the shapes by the gene `CST3` to see the 'hotspot'.
4. In the `napari` layer controls (bottom-left), click the 'Add shapes layer' button (it looks like a polygon).
5. Select the polygon tool from the toolbar at the top-left of the layer controls.
6. Draw a polygon that encloses the high-expression `CST3` region.
7. **Do not close Napari yet!** We will use the viewer object to extract the coordinates of the polygon you just drew.

In [ ]:
# This will create a viewer object that we can interact with programmatically
viewer = nsd.Interactive(sdata_visium)
# After you draw your polygon, leave the Napari window open and run the next cell.

### Extracting Your Annotation

Now that you've drawn a polygon in Napari, we can get its coordinates back into our notebook. The `viewer` object holds all the information about the current Napari session.

**Note:** For reproducibility, if you didn't draw a polygon, the code below will use a pre-defined one. But it's more fun to use your own!

In [ ]:
# In this cell, we will extract the polygon you just drew in Napari.

# The code assumes that the NEW shapes layer you created is now the
# last (i.e., topmost) layer in the Napari layer list.
try:
    roi_layer = viewer._viewer.layers[-1]

    # --- THE FIX IS HERE ---
    # We now use 'isinstance' to check if the layer is a Shapes layer.
    from napari.layers import Shapes

    if isinstance(roi_layer, Shapes) and len(roi_layer.data) > 0:
        # Get the polygon you drew (we'll just take the first one if you drew multiple)
        my_polygon = Polygon(roi_layer.data[0])
        print("Successfully extracted polygon from Napari!")
    else:
        # This will happen if you didn't draw anything on the new layer.
        raise IndexError("The topmost layer is not a Shapes layer or is empty.")

except IndexError:
    # This block runs if no new layer was added, or if nothing was drawn.
    print("Could not extract a drawn polygon. Using a pre-defined example ROI.")
    # Fallback to a pre-defined polygon for the demo
    coords = [
        (722, 443),
        (705, 438),
        (688, 430),
        (655, 407),
        (642, 390),
        (630, 372),
        (620, 353),
        (612, 335),
        (605, 295),
        (603, 279),
        (603, 259),
        (605, 240),
        (618, 222),
        (658, 179),
        (690, 157),
        (705, 149),
        (723, 140),
        (757, 120),
        (775, 115),
        (817, 112),
        (837, 114),
        (857, 114),
        (895, 117),
        (935, 125),
        (955, 134),
        (970, 144),
        (1000, 170),
        (1015, 180),
        (1028, 192),
        (1045, 202),
        (1051, 219),
        (1055, 235),
        (1055, 257),
        (1053, 277),
        (1050, 293),
        (1030, 360),
        (1023, 377),
        (1018, 393),
        (1003, 407),
        (995, 423),
        (985, 438),
        (968, 445),
        (951, 450),
        (928, 455),
        (891, 458),
        (852, 458),
        (752, 450),
        (722, 443),
    ]
    my_polygon = Polygon(coords)


# You can now close the Napari window
viewer._viewer.window.close()

In [ ]:
print(my_polygon)

### Querying Data with the ROI

With our polygon defined, we can use `spatialdata.polygon_query()` to select only the Visium spots that fall within its boundaries. This is a powerful way to subset your data for targeted analysis.

In [ ]:
from spatialdata import polygon_query
from shapely.geometry import Polygon


# Convert 3D to 2D polygon
def force_2d_polygon(polygon):
    if polygon.has_z:
        coords_2d = [(x, y) for x, y, z in polygon.exterior.coords]
        return Polygon(coords_2d)
    return polygon


# Convert your polygon
my_polygon_2d = force_2d_polygon(my_polygon)

# Use "global" coordinate system (not "downscaled_hires")
filtered_sdata = polygon_query(
    sdata_visium,
    polygon=my_polygon_2d,
    target_coordinate_system="global",  # Changed this!
)

print(
    "Original number of spots:",
    len(sdata_visium.shapes["CytAssist_FFPE_Protein_Expression_Human_Glioblastoma"]),
)
print(
    "Number of spots in ROI:",
    len(filtered_sdata.shapes["CytAssist_FFPE_Protein_Expression_Human_Glioblastoma"]),
)

### Annotating the Original Data

Finally, let's use our selection to create a new column in our original `AnnData` table to label the spots inside our ROI. This is useful for differential expression analysis or visualization.

In [ ]:
# Get the names (indices) of the spots that were selected
spots_in_roi = filtered_sdata.tables["table"].obs.index

# Add a new column to the original anndata table
sdata_visium.tables["table"].obs["roi_annotation"] = "Outside ROI"
sdata_visium.tables["table"].obs.loc[spots_in_roi, "roi_annotation"] = "CST3 Hotspot"

# Convert to a categorical for plotting
sdata_visium.tables["table"].obs["roi_annotation"] = (
    sdata_visium.tables["table"].obs["roi_annotation"].astype("category")
)

Now we can visualize this new annotation using `spatialdata-plot`.

In [ ]:
sdata_visium.pl.render_shapes(
    color="roi_annotation",
    fill_alpha=0.7
).pl.show("downscaled_hires")

In this notebook, we've learned a critical skill: how to bridge interactive visualization with programmatic analysis. 

Next, we will perform a full-scale downstream analysis to identify cell types from scratch using `scanpy`.